In [ ]:
原文網址:
https://tlyu0419.github.io/2019/10/30/%E5%A6%82%E4%BD%95%E9%80%8F%E9%81%8E%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2%E6%8A%93%E5%8F%96%E4%B8%AD%E6%99%82%E9%9B%BB%E5%AD%90%E5%A0%B1%E6%96%B0%E8%81%9E/#more

# 載入使用套件

In [10]:
!pip install tomorrow

In [4]:
# 爬蟲類套件
import requests
import json
from bs4 import BeautifulSoup
# 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
#from tomorrow import threads  #錯誤
# 資料整理類套件
import pandas as pd
import re
from datetime import datetime

# 其他
import os

# 擷取一篇新聞

In [1]:
def GetNews_chinatimes(response):
    soup = BeautifulSoup(response.text)
    url = soup.find('link')['href']
    ndf = pd.DataFrame(data = [{'TITLE':soup.find('h1', attrs={'class':'article-title'}).text,
                                'TIME':datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'],'%Y-%m-%dT%H:%M:%S+08:00'),
                                'CATEGORY':soup.find('meta',attrs={'property':'article:section'})['content'],
                                'DESCRIPTION':soup.find('meta',attrs={'name':'description'})['content'],
                                'CONTENT':'\n'.join(i.text for i in soup.find('div',attrs={'class':'article-body'}).find_all('p')),
                                'KEYWORDS':soup.find('meta',{'name':'news_keywords'})['content'],
                                'FROM':soup.find('meta',{'name':'publisher'})['content'],
                                'LINK':soup.find('meta', {'property':'og:url'})['content']}],
                       columns = ['TITLE', 'TIME', 'CATEGORY', 'DESCRIPTION', 'CONTENT','KEYWORDS', 'FROM', 'LINK']) 
    return ndf

In [5]:
url = 'https://www.chinatimes.com/newspapers/20191028000424-260118?chdtv'
resp = requests.get(url)
GetNews_chinatimes(resp)

,TITLE,TIME,CATEGORY,DESCRIPTION,CONTENT,KEYWORDS,FROM,LINK
0,直球對決 韓接招青年辣提問！可以討厭我 但請聽我的政策,2019-10-28 04:10:19,政治要聞,國民黨總統參選人韓國瑜昨晚在台北舉辦青年論壇，標榜「走出同溫層」，果然在場青年提問辛辣，毫不...,國民黨總統參選人韓國瑜昨晚在台北舉辦青年論壇，標榜「走出同溫層」，果然在場青年提問辛辣，毫不...,"韓國瑜,國民黨,2020大選",中時電子報,https://www.chinatimes.com/newspapers/20191028...


# 擷取特定關鍵詞新聞的連結清單

In [21]:
def GetLinks_chinatimes(response):
    links = []
    soup = BeautifulSoup(response.text)
    for i in soup.find_all('h3'):
        url = i.find('a')['href']
        links.append(url)
    return links

In [23]:
url = 'https://www.chinatimes.com/search/英國'
resp = requests.get(url)
GetLinks_chinatimes(resp)

['https://www.chinatimes.com/newspapers/20191210000755-260109',
 'https://www.chinatimes.com/realtimenews/20191210000009-260408',
 'https://www.chinatimes.com/realtimenews/20191209000917-260408',
 'https://www.chinatimes.com/opinion/20191209003325-262104',
 'https://www.chinatimes.com/realtimenews/20191209000995-260408',
 'https://www.chinatimes.com/realtimenews/20191209001331-260408',
 'https://www.chinatimes.com/realtimenews/20191208002943-260407',
 'https://www.chinatimes.com/newspapers/20191208000097-260301',
 'https://www.chinatimes.com/realtimenews/20191207003114-260408',
 'https://www.chinatimes.com/realtimenews/20191207001601-260409',
 'https://www.chinatimes.com/realtimenews/20191207002044-260409',
 'https://www.chinatimes.com/realtimenews/20191207000016-260408',
 'https://www.chinatimes.com/realtimenews/20191206002037-260408',
 'https://www.chinatimes.com/realtimenews/20191205001064-260408',
 'https://www.chinatimes.com/realtimenews/20191205004939-260408',
 'https://www.china

In [ ]:
## 開啟多線程功能
* 只要組合以上兩個函數，其實就已經可以實現自動化的爬蟲作業了，具體方式其實就是逐一的把新聞連結清單放入 for 迴圈中，並透過 GetNews_chinatimes 函數解析新聞就可以了
* 這樣m的方式在少量資料的時候還可以，但當我們需要抓上千、萬篇新聞時，「逐一」這件事請就顯得非常沒有效率。那有什麼辦法解決呢? 答案是我們可以運用多核心多線程的功能同時抓取多篇新聞內容，只需要運用 tomorrow 套件就能讓開啟多線程變得非常簡單，使用方式如下

In [ ]:
使用提醒：

* threadms 數開得太高有可能會導致被網站封鎖 IP 而禁止連線。
* 具體要設定多少 threads 數才不會被鎖?這需要慢慢嘗試過後能才知道，每個網站的反爬蟲機制不太一致。
* 建議在抓資料時使用手機分享的網路，因為被封鎖 IP 狀況發生時只需要開啟飛航模式再關閉，就會替換成新 IP 而解除封鎖囉!

In [0]:
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

# 組合應用

In [0]:
def CrawlNews_chinatimes(keywords, pages):
    # 截取多個分頁的新聞連結
    links = []
    for i in range(pages):
        url = 'https://www.chinatimes.com/search/{}?page={}'.format(keywords, i+1)
        resp = requests.get(url)
        links += GetLinks_chinatimes(resp)   
    print('There are {} links in pages.'.format(len(links)))

    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    list_of_dataframes = []
    for response in responses:
        try:
            ndf = GetNews_chinatimes(response)
            list_of_dataframes.append(ndf)
        except:
            pass
    df = pd.concat(list_of_dataframes, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

In [0]:
%%time
# 可以自行替換查詢的關鍵字，另外需要更多新聞的人也可以把 pages 的數值調高)
df = CrawlNews_chinatimes(keywords='美國', pages=10)

There are 200 links in pages.
There are 200 News in DataFrame.
CPU times: user 8.82 s, sys: 410 ms, total: 9.23 s
Wall time: 17.4 s


In [0]:
df

,TITLE,TIME,CATEGORY,DESCRIPTION,CONTENT,KEYWORDS,FROM,LINK
0,美國多重收益平衡基金 抗震,2019-10-30 04:10:12,投資理財,全球投資環境歷經2017年到2018年的穩健復甦後，2019年經濟成長出現雜音，從美國聯準會...,全球投資環境歷經2017年到2018年的穩健復甦後，2019年經濟成長出現雜音，從美國聯準會...,"投資,降息,美國",中時電子報,https://www.chinatimes.com/newspapers/20191030...
1,成大鏈結美國新創團隊,2019-10-30 04:10:08,產業．科技,國立成功大學23日在美國波士頓與跨國創投公司EBO Global Innovation Ca...,國立成功大學23日在美國波士頓與跨國創投公司EBO Global Innovation Ca...,"美國,創投",中時電子報,https://www.chinatimes.com/newspapers/20191030...
2,美國NAE院士蕭美琛 北科大首位女名譽博士,2019-10-30 04:10:16,產業特刊,臺北科大創校108周年校慶26日在該校盛大舉行，北科大校友、美國國家工程學院（NAE）新科院...,臺北科大創校108周年校慶26日在該校盛大舉行，北科大校友、美國國家工程學院（NAE）新科院...,美國,中時電子報,https://www.chinatimes.com/newspapers/20191030...
3,分析師預估 美國第3季GDP成長1.6％,2019-10-30 13:26:23,國際,路透報導，美國經濟在第3季進一步趨緩，主要受到消費者支出溫和與企業投資衰退所累，促使聯準會可...,路透報導，美國經濟在第3季進一步趨緩，主要受到消費者支出溫和與企業投資衰退所累，促使聯準會可...,"美國,gdp,經濟,降息,fed",中時電子報,https://www.chinatimes.com/realtimenews/201910...
4,無色覺醒》賴岳謙：美國執意敵視中國！中國將成美國惡夢！,2019-10-30 08:30:00,無色覺醒,歡迎收看《無色覺醒》第462集播出，由主講人賴岳謙為觀眾分析：「美國執意敵視中國！中國將成美...,歡迎收看《無色覺醒》第462集播出，由主講人賴岳謙為觀眾分析：「美國執意敵視中國！中國將成美...,"無色覺醒,賴岳謙,中時電子報",中時電子報,https://www.chinatimes.com/opinion/20191030000...
...,...,...,...,...,...,...,...,...
195,庫德族悲歌給台灣的啟示,2019-10-24 04:10:00,論壇廣場,分布在土耳其、敘利亞、伊朗、伊拉克邊境交界處的庫德族，自第一次大戰以來，便存有獨立建國的理想...,分布在土耳其、敘利亞、伊朗、伊拉克邊境交界處的庫德族，自第一次大戰以來，便存有獨立建國的理想...,"敘利亞,美國,庫德族",中時電子報,https://www.chinatimes.com/newspapers/20191024...
196,陸外交部籲美 停止「限制兩國科研交流」的作為,2019-10-24 21:28:00,財經,美國智庫戰略與國際問題研究中心發布了《戰略競爭時代的科研合作》報告，報告指出，美國以國家安全...,美國智庫戰略與國際問題研究中心發布了《戰略競爭時代的科研合作》報告，報告指出，美國以國家安全...,"合作,國家,美國,科學,研究",中時電子報,https://www.chinatimes.com/realtimenews/201910...
197,土耳其同意永久停火 川普宣布撤銷制裁,2019-10-24 08:20:47,財經,美國總統川普23日宣布，土耳其當局已停止在敘利亞的攻擊行動並願意「永久停火」，因此美國將取消...,美國總統川普23日宣布，土耳其當局已停止在敘利亞的攻擊行動並願意「永久停火」，因此美國將取消...,"制裁,美國,土耳其,川普,停火",中時電子報,https://www.chinatimes.com/realtimenews/201910...
198,淨進口降至新低 油價走高,2019-10-24 18:53:58,財經,上周美國原油進口每日下降43.8萬桶，同時出口每日增加43.5萬桶，使得淨進口降至每日217...,上周美國原油進口每日下降43.8萬桶，同時出口每日增加43.5萬桶，使得淨進口降至每日217...,"美國,原油,進口,央行,歷史",中時電子報,https://www.chinatimes.com/realtimenews/201910...


# 儲存結果

## 串接 Google Drive 空間

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 保存資料
- 資料會保存在 Google Drive 上 Colab Notebooks/WebCrawler/TW_News/News 的資料夾空間中

In [0]:
import os
path = '/content/drive/My Drive/Colab Notebooks/WebCrawler/TW_News/News'
# 如果不存在這個資料夾就新建這個資料夾
if not os.path.exists(path):
    os.makedirs(path)
df.to_excel(path + '/News_chinatimes.xlsx')